In [ ]:
from Adafruit_AMG88xx import Adafruit_AMG88xx
import pygame
import os
import math
import time
import cv2
import numpy as np
import datetime
from PIL import Image
from scipy.interpolate import griddata
from colour import Color

In [ ]:
# pygame display capture
def Capture(display,img_num): # (pygame Surface, String, tuple, tuple)
    name = "image" + str(img_num)
    pygame.image.save(display, "/home/pi/Pictures/{}.PNG".format(name))
    

In [ ]:
#low range of the sensor (this will be blue on the screen)
MINTEMP = 24

#high range of the sensor (this will be red on the screen)
MAXTEMP = 28

#how many color values we can have
COLORDEPTH = 1024

os.putenv('SDL_FBDEV', '/dev/fb1')
pygame.init()

#initialize the sensor
sensor = Adafruit_AMG88xx()

points = [(math.floor(ix / 8), (ix % 8)) for ix in range(0, 64)]
grid_x, grid_y = np.mgrid[0:7:32j, 0:7:32j]

#sensor is an 8x8 grid so lets do a square
height = 240
width = 240

#the list of colors we can choose from
blue = Color("indigo")
colors = list(blue.range_to(Color("red"), COLORDEPTH))

#create the array of colors
colors = [(int(c.red * 255), int(c.green * 255), int(c.blue * 255)) for c in colors]

displayPixelWidth = width / 30
displayPixelHeight = height / 30

size = (displayPixelWidth, displayPixelHeight)

lcd = pygame.display.set_mode((width, height))

lcd.fill((255,0,0))

pygame.display.update()
pygame.mouse.set_visible(False)

lcd.fill((0,0,0))
pygame.display.update()

#some utility functions
def constrain(val, min_val, max_val):
    return min(max_val, max(min_val, val))

def map(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

#let the sensor initialize
time.sleep(.1)

img_num = 0
sec = 0

In [ ]:
while(1):
    #read the pixels
    pixels = sensor.readPixels()
    pixels = [map(p, MINTEMP, MAXTEMP, 0, COLORDEPTH - 1) for p in pixels]

    #perdorm interpolation
    bicubic = griddata(points, pixels, (grid_x, grid_y), method='cubic')
    
    #draw everything
    for ix, row in enumerate(bicubic):
        for jx, pixel in enumerate(row):
            pygame.draw.rect(lcd, colors[constrain(int(pixel), 0, COLORDEPTH- 1)],
                             (displayPixelHeight * ix, displayPixelWidth * jx, displayPixelHeight, displayPixelWidth))
    pygame.display.update()
    
    sec += 1
    time.sleep(0.1)
    
    if sec == 2:
        sec = 0 
        img_num = img_num + 1    
        Capture(lcd,img_num)